In [ ]:
#pip install tensorflow==2.10
#pip install python-binance
#pip install pandas
#pip install -U scikit-learn

In [ ]:
pip install -U scikit-learn

In [2]:
#Setup Data
apikey  = 'QHOA07WHc6ZkNS2A9i4bTcTbYuOpEqLI3eHu6iWp02gDOnmwMFpQsSQDO4nytISY'
secret = 'yvHF7zGHB9CVj8cSHQb0j8cZdjhfNSdJ7FcWJuvIfVlNcCh4aT6UneTJRjcokzH9'

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import warnings
from termcolor import colored
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

#Autenticate
client = Client(apikey, secret)

In [ ]:
#Historical Data
file_path = r'C:\Users\DJKlaKunG\Desktop\Personal Document\GitHub\CNN-Bi-LSTM-Research\training-data\hist_data.csv'

if os.path.exists(file_path):
    print("Requestin Old Data")
    hist_df = pd.read_csv(file_path)
    print("Requestin Old Data - Complete")
else:
    print("Requestin New Data")
    historical = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_15MINUTE, '1 Jan 2019')
    hist_df = pd.DataFrame(historical)
    hist_df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                        'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
    hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
    hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000, unit='s')
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']
    hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)
    print("Requestin New Data - Complete")


#Preprocessing Data
hist_df.to_csv(file_path, index=False)
print("Preprocessing Data - Complete")

In [ ]:
#Calculate the Nessary Value then appear in the CSV file
warnings.filterwarnings('ignore')
# 1. Calculate Gain and Loss
hist_df['Gain'] = hist_df['Close'].diff().apply(lambda x: x if x > 0 else 0)
hist_df['Loss'] = hist_df['Close'].diff().apply(lambda x: -x if x < 0 else 0)

# 2. Calculate Average Gain and Average Loss for the last 14 bars
hist_df['Avg_Gain_14'] = hist_df['Gain'].rolling(window=14).mean()
hist_df['Avg_Loss_14'] = hist_df['Loss'].rolling(window=14).mean()

# 3. Calculate ADX for 7 bars
# First, we need to calculate the True Range, +DM, and -DM
hist_df['HL'] = hist_df['High'] - hist_df['Low']
hist_df['HPC'] = (hist_df['High'] - hist_df['Close']).shift(1)
hist_df['LPC'] = (hist_df['Close'].shift(1) - hist_df['Low'])

# True Range
hist_df['TR'] = hist_df[['HL', 'HPC', 'LPC']].max(axis=1)

# Positive Directional Movement (+DM) and Negative Directional Movement (-DM)
hist_df['+DM'] = ((hist_df['High'] - hist_df['High'].shift(1)) > 
                 (hist_df['Low'].shift(1) - hist_df['Low'])).astype(int) * hist_df['HL']
hist_df['-DM'] = ((hist_df['Low'].shift(1) - hist_df['Low']) > 
                 (hist_df['High'] - hist_df['High'].shift(1))).astype(int) * hist_df['HL']

# Smoothed True Range and Directional Movements
hist_df['Smoothed_TR'] = hist_df['TR'].rolling(window=7).sum()
hist_df['Smoothed_+DM'] = hist_df['+DM'].rolling(window=7).sum()
hist_df['Smoothed_-DM'] = hist_df['-DM'].rolling(window=7).sum()

# Directional Indicators
hist_df['+DI'] = 100 * hist_df['Smoothed_+DM'] / hist_df['Smoothed_TR']
hist_df['-DI'] = 100 * hist_df['Smoothed_-DM'] / hist_df['Smoothed_TR']

# DX (Directional Movement Index)
hist_df['DX'] = 100 * (hist_df['+DI'] - hist_df['-DI']).abs() / (hist_df['+DI'] + hist_df['-DI'])

# ADX (Average Directional Index)
hist_df['ADX_7'] = hist_df['DX'].rolling(window=7).mean()

# 4. Calculate EMA for 7 bars
hist_df['EMA_7'] = hist_df['Close'].ewm(span=7, adjust=False).mean()

# Drop intermediate columns used for calculations
drop_cols = ['HL', 'HPC', 'LPC', 'TR', '+DM', '-DM', 'Smoothed_TR', 'Smoothed_+DM', 'Smoothed_-DM', '+DI', '-DI', 'DX']
hist_df.drop(columns=drop_cols, inplace=True)

# Save the dataframe with calculated values to the specified file path
hist_df.to_csv(file_path, index=False)

print("Calculate Necessary data - Complete")



In [ ]:
#Calculate the Stationary Wavelet Transform (SWT)
import pywt
#Redefine the list of columns to transform
columns_to_transform = ['Close', 'Gain', 'Loss', 'EMA_7','ADX_7']

# Define the Fourier Transformation function again
def apply_swt(series):
    # Apply Stationary Wavelet Transform (SWT) to the series
    coeffs = pywt.swt(series, wavelet='db1', level=1)
    # We'll use the approximation coefficients for our analysis
    return coeffs[0][0]

# Apply SWT to the specified columns in the dataset
for col in columns_to_transform:
    hist_df[f'{col}_SWT'] = apply_swt(hist_df[col])

# Save the dataframe with calculated values to the specified file path
hist_df.to_csv(file_path, index=False)

print("Calculate Necessary data - Complete")

In [ ]:
#Data Preparation: Shape the data into appropriate sequences for time series forecasting.

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Select the transformed Close price as our feature
feature_column = 'Close_SWT'
target_column = 'Close_SWT'

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(hist_df[[feature_column]])

# Create sequences of length 15 (14 features + 1 target)
X, y = [], []
sequence_length = 15

for i in range(len(scaled_data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length-1])
    y.append(scaled_data[i+sequence_length-1])

X, y = np.array(X), np.array(y)

# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Reshape the input to be [samples, time steps, features] which is required for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

X_train.shape, X_test.shape


In [ ]:
# Building the CNN-Bi-LSTM Model:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError

# Hyperparameters
LOOKBACK = 14
EPOCHS = 100
BATCH_SIZE = 256
LEARNING_RATE = 0.0001
KERNEL_SIZE = 3
FILTERS = 64
POOL_SIZE = 2
LSTM_UNITS1 = 50
LSTM_UNITS2 = 50
ACTIVATION_FUNC = 'relu'
LOSS_FUNC = 'mean_squared_error'
PERFORMANCE_METRIC = RootMeanSquaredError(name='rmse')

# Modified model with an additional CNN layer

model = Sequential()

# First CNN layer
model.add(Conv1D(filters=FILTERS, kernel_size=KERNEL_SIZE, activation=ACTIVATION_FUNC, input_shape=(LOOKBACK, 1)))
model.add(MaxPooling1D(pool_size=POOL_SIZE))

# Bi-LSTM layers
model.add(Bidirectional(LSTM(LSTM_UNITS1, return_sequences=True)))
model.add(Bidirectional(LSTM(LSTM_UNITS2)))

# Dense layer for prediction
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss=LOSS_FUNC, metrics=[PERFORMANCE_METRIC])

# Display the modified model summary
model.summary()


In [ ]:
#Training In Progress..

import time

print("Training In Progress...")

# Record the start time
start_time = time.time()

history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), verbose=1)

# Record the end time
end_time = time.time()

# Calculate total training time and average time per step
total_time = end_time - start_time
avg_time_per_step = total_time / (len(X_train) // BATCH_SIZE * EPOCHS)  # Using '//' for integer division

print("\nTraining Completed!")
print(f"Total Training Time: {total_time:.2f} seconds")
print(f"Average Time Per Step: {avg_time_per_step:.4f} seconds")


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
